In [1]:
# !pip install Basemap

In [2]:
# !pip install haversine

In [3]:
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [33]:
import glob
import os
import pandas as pd
import numpy as np
from haversine import haversine
import matplotlib
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') 
import warnings
warnings.filterwarnings(action='ignore')

In [5]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


# 데이터 전처리

**2000-2021년 6-10월 데이터**

In [6]:
path = '/content/gdrive/My Drive/hot/1year/'
File_list = glob.glob(os.path.join(path, 'OBS*.csv'))
# print(File_list)

allData = [] # 읽어 들인 excel파일 내용을 저장할 빈 리스트를 하나 만든다
for file in File_list:
    df = pd.read_csv(file,encoding='CP949') # for구문으로 excel파일들을 읽어 들인다
    allData.append(df)
data = pd.concat(allData, axis=0, ignore_index=True).sort_values(['지점명','일시'])
data['지점명'] = data['지점명'].str.strip()

data

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),일강수량(mm),최대 순간 풍속(m/s),최대 순간풍속 시각(hhmi),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
1490298,303,가거도,2001-06-01,19.6,16.8,59.0,22.2,1010.0,0.0,11.0,2210.0,2.2,228.0
1490299,303,가거도,2001-06-02,20.2,17.5,2148.0,22.7,1213.0,0.0,12.3,1408.0,2.1,281.0
1490300,303,가거도,2001-06-03,20.7,19.0,2338.0,23.6,1324.0,0.0,12.7,2111.0,2.3,314.0
1490301,303,가거도,2001-06-04,18.8,17.0,2016.0,21.3,1237.0,0.0,9.4,44.0,0.8,289.0
1490302,303,가거도,2001-06-05,16.9,15.8,1647.0,19.7,1.0,10.0,12.5,1850.0,1.9,91.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17798,536,횡성,2021-10-27,10.5,4.2,2358.0,19.9,1329.0,0.0,6.8,1538.0,1.0,213.8
17799,536,횡성,2021-10-28,8.9,0.8,620.0,19.5,1447.0,0.0,3.4,1308.0,0.8,213.8
17800,536,횡성,2021-10-29,10.3,3.8,624.0,19.7,1522.0,0.0,4.3,1051.0,0.9,19.7
17801,536,횡성,2021-10-30,9.8,5.4,652.0,17.9,1302.0,0.0,3.7,1525.0,0.7,205.3


**2000년부터 존재하는 지점만 추출**

In [7]:
path = '/content/gdrive/My Drive/hot/1year/'
dat = pd.read_csv(path+'OBS_AWS_DD_20220316142920.csv',encoding='CP949')
#dat.drop(data.columns[0],inplace=True,axis=1)
dat['지점명'] = dat['지점명'].str.strip()
dat

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),일강수량(mm),최대 순간 풍속(m/s),최대 순간풍속 시각(hhmi),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,116,관악(레),2000-06-01,16.7,13.9,1.0,21.3,1348.0,0.0,7.1,1424.0,1.1,0.0
1,116,관악(레),2000-06-02,18.5,14.3,153.0,23.2,1313.0,0.0,7.1,1.0,0.5,298.8
2,116,관악(레),2000-06-03,21.5,16.6,555.0,27.3,1526.0,0.0,7.1,11.0,0.2,0.0
3,116,관악(레),2000-06-04,21.9,17.1,535.0,28.6,1610.0,0.0,9.0,42.0,0.1,0.0
4,116,관악(레),2000-06-05,21.6,18.8,2027.0,26.0,1355.0,0.0,11.1,2215.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
57928,951,내장산,2000-10-28,11.0,6.8,28.0,16.0,1316.0,0.0,4.6,1345.0,0.2,297.6
57929,951,내장산,2000-10-29,9.1,4.4,2342.0,13.1,1407.0,1.0,5.4,1521.0,0.2,231.1
57930,951,내장산,2000-10-30,6.8,1.6,2359.0,15.8,1324.0,0.0,5.4,1418.0,0.3,271.6
57931,951,내장산,2000-10-31,8.8,0.9,505.0,16.7,1108.0,0.0,3.9,1004.0,0.2,124.5


In [8]:
dat = data[data['지점명'].isin(dat['지점명'].unique())].reset_index(drop=True)
dat

,지점,지점명,일시,평균기온(°C),최저기온(°C),최저기온 시각(hhmi),최고기온(°C),최고기온 시각(hhmi),일강수량(mm),최대 순간 풍속(m/s),최대 순간풍속 시각(hhmi),평균 풍속(m/s),최대 순간 풍속 풍향(deg)
0,647,가곡,2000-06-01,20.3,11.8,602.0,29.5,1403.0,0.0,5.3,1333.0,0.1,89.6
1,647,가곡,2000-06-02,21.8,15.1,436.0,28.3,1617.0,0.0,3.1,1327.0,0.2,183.0
2,647,가곡,2000-06-03,22.9,17.6,411.0,28.3,1429.0,0.0,5.8,1054.0,0.9,18.6
3,647,가곡,2000-06-04,22.0,14.7,531.0,31.1,1445.0,0.0,9.6,1346.0,1.1,60.4
4,647,가곡,2000-06-05,21.8,11.5,452.0,31.0,1425.0,0.0,7.5,1256.0,1.1,19.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1286102,536,횡성,2021-10-27,10.5,4.2,2358.0,19.9,1329.0,0.0,6.8,1538.0,1.0,213.8
1286103,536,횡성,2021-10-28,8.9,0.8,620.0,19.5,1447.0,0.0,3.4,1308.0,0.8,213.8
1286104,536,횡성,2021-10-29,10.3,3.8,624.0,19.7,1522.0,0.0,4.3,1051.0,0.9,19.7
1286105,536,횡성,2021-10-30,9.8,5.4,652.0,17.9,1302.0,0.0,3.7,1525.0,0.7,205.3


In [9]:
len(dat['지점명'].unique())

387

In [10]:
new = dat[['지점명','일시','최고기온(°C)']]
print(len(new[new['최고기온(°C)'].isnull()]['일시'].unique()))
new

3028


,지점명,일시,최고기온(°C)
0,가곡,2000-06-01,29.5
1,가곡,2000-06-02,28.3
2,가곡,2000-06-03,28.3
3,가곡,2000-06-04,31.1
4,가곡,2000-06-05,31.0
...,...,...,...
1286102,횡성,2021-10-27,19.9
1286103,횡성,2021-10-28,19.5
1286104,횡성,2021-10-29,19.7
1286105,횡성,2021-10-30,17.9


In [13]:
set(new[new['일시']=='2000-06-02']['지점명'].unique())^set(new[new['일시']=='2000-06-03']['지점명'].unique())

{'진달래밭'}

In [16]:
pp = pd.DataFrame(index=dat['지점명'].unique(),columns=['최고기온(°C)'])
pp

,최고기온(°C)
가곡,NaN
가덕,NaN
가덕도,NaN
가산,NaN
가평북면,NaN
...,...
화천,NaN
화촌,NaN
황성,NaN
황전,NaN


In [25]:
dk = []
for i in new['일시'].unique():
  dk.append(pp.combine_first(new[new['일시']==i][['지점명','최고기온(°C)']].set_index('지점명'))['최고기온(°C)'])

In [28]:
dk[:2]

[가곡      29.5
 가덕      31.6
 가덕도     19.9
 가산      29.9
 가평북면    28.5
         ... 
 화천      28.0
 화촌      28.0
 황성      24.1
 황전      28.0
 횡성      29.3
 Name: 최고기온(°C), Length: 387, dtype: object, 가곡      28.3
 가덕      29.8
 가덕도     22.7
 가산      26.8
 가평북면    29.7
         ... 
 화천      30.0
 화촌      30.2
 황성      25.9
 황전      27.5
 횡성      29.7
 Name: 최고기온(°C), Length: 387, dtype: object]

In [29]:
np.array(dk).shape

(3366, 387)

**가장 최근에 측정시작한 위경도 추출**

In [30]:
meta = pd.read_csv(path+'../META_관측지점정보.csv',encoding='CP949')

meta = meta.sort_values(['지점명','시작일'])

meta = meta.drop_duplicates(subset='지점명', keep='last')[['지점명','위도','경도']].reset_index(drop=True)
meta

,지점명,위도,경도
0,13번국도(완도대교),34.3942,126.6468
1,309호선 학의JC남측,37.3892,127.0022
2,31번국도(어평재),37.1200,128.8800
3,322호선 전곡항입구,37.1911,126.6536
4,38번국도(나한정역),37.1983,128.0329
...,...,...,...
5182,흥해읍,36.1071,129.3417
5183,희망,37.6970,127.8888
5184,희천,40.1667,126.2500
5185,흰바우산,38.3115,127.7430


In [31]:
x = pd.merge(new,meta,on='지점명')
x

,지점명,일시,최고기온(°C),위도,경도
0,가곡,2000-06-01,29.5,36.1250,127.6924
1,가곡,2000-06-02,28.3,36.1250,127.6924
2,가곡,2000-06-03,28.3,36.1250,127.6924
3,가곡,2000-06-04,31.1,36.1250,127.6924
4,가곡,2000-06-05,31.0,36.1250,127.6924
...,...,...,...,...,...
1286102,횡성,2021-10-27,19.9,37.4876,127.9724
1286103,횡성,2021-10-28,19.5,37.4876,127.9724
1286104,횡성,2021-10-29,19.7,37.4876,127.9724
1286105,횡성,2021-10-30,17.9,37.4876,127.9724


In [32]:
xx = x.drop_duplicates(subset='지점명', keep='last')[['지점명','위도','경도']].reset_index(drop=True)
xx

,지점명,위도,경도
0,가곡,36.1250,127.6924
1,가덕,36.5461,127.5239
2,가덕도,34.9931,128.8314
3,가산,37.8486,127.1864
4,가평북면,37.8981,127.5522
...,...,...,...
382,화천,38.0964,127.7029
383,화촌,37.7871,127.9838
384,황성,35.8630,129.2093
385,황전,35.1007,127.4274


가까운 지점 호출

In [34]:
d={}
for i in range(len(xx)):
  s = []
  for j in range(len(xx)):
    if i==j:
      s.append(10**10)
    else:
      start = (xx['위도'].iloc[i], xx['경도'].iloc[i]) 
      goal = (xx['위도'].iloc[j], xx['경도'].iloc[j])
      s.append(haversine(start,goal))
  d[xx['지점명'].iloc[i]]=xx['지점명'].iloc[s.index(min(s))]

print(len(d))

387


In [35]:
d['하남']

'구리'

In [40]:
du = pp.combine_first(new[new['일시']=='2000-06-01'][['지점명','최고기온(°C)']].set_index('지점명'))['최고기온(°C)']
for i in du[du.isnull()].index:
  du[i]=du[d[i]]
  print(du[i])

27.6
18.5
28.0
21.0
18.6
25.3
27.8
25.8
nan
21.0
nan
27.2
27.1
27.1
30.2


In [36]:
def area(spot):
  d={}
  s = []
  for i in range(len(xx)):
    if spot==xx.iloc[i]['지점명']:
      s.append(10**10)
    else:
      start = (xx[xx['지점명']==spot]['위도'].values[0], xx[xx['지점명']==spot]['경도'].values[0]) 
      goal = (xx['위도'].iloc[i], xx['경도'].iloc[i])
      s.append(haversine(start,goal))
  return xx['지점명'].iloc[s.index(min(s))]

print(area('하남'))

구리
